In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import librosa
import os
from IPython.display import Audio
import matplotlib.pyplot as plt
import soundfile as sf

In [ ]:
from models.tts.gpt_tts.gpt_tts import GPTTTS
from models.tts.gpt_tts.g2p_old_en import process, PHPONE2ID
from g2p_en import G2p
from models.codec.codec_latent.codec_latent import LatentCodecEncoder, LatentCodecDecoderWithTimbre
from models.codec.amphion_codec.codec import CodecEncoder, CodecDecoder
from utils.util import load_config

In [ ]:
cfg = load_config("egs/tts/LaCoTTS/exp_config_base.json")
print(cfg)

# Audio Tokenizer: Convert Speech to Token

In [ ]:
wav_codec_enc = CodecEncoder(
    cfg=cfg.model.wav_codec.encoder
)
wav_codec_dec = CodecDecoder(
    cfg=cfg.model.wav_codec.decoder
)

In [ ]:
latent_codec_enc = LatentCodecEncoder(
    cfg=cfg.model.latent_codec.encoder
)
latent_codec_dec = LatentCodecDecoderWithTimbre(
    cfg=cfg.model.latent_codec.decoder
)

In [ ]:
wav_codec_enc.load_state_dict(torch.load("ckpts/wav_codec/wav_codec_enc.bin"))
wav_codec_dec.load_state_dict(torch.load("ckpts/wav_codec/wav_codec_dec.bin"))
latent_codec_enc.load_state_dict(torch.load("ckpts/latent_codec/latent_codec_enc.bin"))
latent_codec_dec.load_state_dict(torch.load("ckpts/latent_codec/latent_codec_dec.bin"))

wav_codec_enc.eval()
wav_codec_dec.eval()
latent_codec_enc.eval()
latent_codec_dec.eval()

wav_codec_enc.cuda()
wav_codec_dec.cuda()
latent_codec_enc.cuda()
latent_codec_dec.cuda()

# requires_grad false
wav_codec_enc.requires_grad_(False)
wav_codec_dec.requires_grad_(False)
latent_codec_enc.requires_grad_(False)
latent_codec_dec.requires_grad_(False)

# Latent Codec Language Model TTS

In [ ]:
gpt_tts = GPTTTS(cfg=cfg.model.gpt_tts)
gpt_tts.load_state_dict(torch.load("ckpts/gpt_tts/latent_codec_gpt_tts.bin"))

In [ ]:
gpt_tts.eval()
gpt_tts.cuda()
gpt_tts.requires_grad_(False)

# Inference

In [ ]:
source_text = "and keeping erernity before the eyes"
target_text = "Come, come returned Hawkeye, uncasing his honest countenance, the better to assure the wavering confidence of his companion. You may see a skin which, if it be not as white as one of the gentle ones, has no tinge of red to it that the winds of the heaven and the sun have not bestowed. Now, let us to business."
source_wav = librosa.load("examples/ref/1.wav", sr=16000)[0]
Audio(source_wav, rate=16000)

In [ ]:
# text tokenize
text = source_text + " " + target_text
g2p = G2p()
txt_struct, txt = process(text, g2p)
phone_seq = [p for w in txt_struct for p in w[1]]
phone_id = [PHPONE2ID[p] for p in phone_seq]
phone_id = torch.LongTensor(phone_id).unsqueeze(0).cuda()
phone_id.shape

In [ ]:
# speech tokenize
prompt_wav = torch.FloatTensor(source_wav).cuda().unsqueeze(0)
# wav to latent
vq_emb = wav_codec_enc(prompt_wav.unsqueeze(1))
vq_emb = latent_codec_enc(vq_emb)
# latent to token
(
    _,
    vq_indices,
    _,
    _,
    _,
    speaker_embedding,
) = latent_codec_dec(vq_emb, vq=True, eval_vq=False, return_spk_embs=True)
prompt_id = vq_indices[0,:,:]
prompt_id.shape

In [ ]:
gen_tokens = gpt_tts.sample_hf(
    phone_id,
    prompt_id,
    max_length=3600,
    temperature=0.9,
    top_k=8192,
    top_p=0.85,
    repeat_penalty=1.0,
    classifer_free_guidance=1.25,   # i find speech speed will be faster if we set cfg > 1.0; if you don't want it, set cfg = 1.0
)

In [ ]:
# gen token to latent
vq_post_emb = latent_codec_dec.vq2emb(gen_tokens.unsqueeze(0))
recovered_latent = latent_codec_dec(
    vq_post_emb, vq=False, speaker_embedding=speaker_embedding
)

In [ ]:
# reconvered latent to wav
recovered_audio = wav_codec_dec(recovered_latent, vq=False)

In [ ]:
sf.write("examples/recon/1.wav", recovered_audio.squeeze().cpu().numpy(), 16000)
Audio("examples/recon/1.wav", rate=16000)